# **Integración de Recuperación con FAISS y Generación con Bloom**

Este script realiza:
- Recuperación de contexto con FAISS.
- Fine-tuning del modelo generativo usando información del OWASP Top 10.
- Generación de respuestas detalladas con un modelo generativo optimizado.


## **Configuración inicial**

Instalamos las bibliotecas necesarias y configuramos las rutas de entrada y salida.

In [1]:
!pip install markdown sentence-transformers faiss-cpu transformers datasets torch accelerate bitsandbytes torch evaluate rouge_score

import os
import re
import json
from markdown import markdown
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset
import numpy as np
import evaluate
import torch
from google.colab import userdata
from torch.quantization import quantize_dynamic
from torch.nn.utils import prune
from transformers import DataCollatorForLanguageModeling, Trainer, AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from transformers.trainer_callback import TrainerCallback
from sklearn.feature_extraction.text import TfidfVectorizer
import time

# Configurar las rutas de entrada y salida
MD_FOLDER = "./md_files"  # Carpeta con archivos Markdown
OUTPUT_JSON = "./owasp_cleaned_dataset.json"  # Dataset unificado
INDEX_FILE = "./indice_faiss.index"  # Archivo FAISS indexado
MODEL_NAME = "bigscience/bloom-560m"  # Modelo de generación de texto
FINE_TUNED_MODEL_PATH = "./fine_tuned_bloom_owasp"  # Ruta para guardar el modelo fine-tuned

In [2]:
!export CUDA_LAUNCH_BLOCKING=1

## **Funciones para procesar archivos Markdown**

Aquí se encuentran las funciones necesarias para procesar los archivos Markdown, limpiar las categorías y contenido, y guardar los datos en un archivo JSON.

In [3]:
def load_json(file_path):
    """Carga un archivo JSON."""
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

def save_json(data, file_path):
    """Guarda un archivo JSON."""
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def clean_text(content):
    """Elimina etiquetas HTML y caracteres innecesarios."""
    content = re.sub(r"<.*?>", "", content)
    content = re.sub(r"\{:.*?\}", "", content)
    content = re.sub(r"\|.*?\|", "", content)
    content = re.sub(r"\n\s*\n", "\n", content)
    content = re.sub(r"\s+", " ", content)
    return content.strip()

def unify_datasets(qa_dataset, faiss_dataset):
    """
    Une y limpia los datasets de QA y FAISS.
    """
    unified_data = []

    # Procesar el dataset de QA
    for entry in qa_dataset:
        question = entry["question"]
        expected = clean_text(entry["context"])
        answer = entry["answers"][0]["text"] if "answers" in entry and entry["answers"] else "No answer provided"

        unified_data.append({
            "content": f"{expected}",
            "category": f"{question}",
            "output_text": answer
        })

    # Procesar el dataset de FAISS
    for entry in faiss_dataset:
        content = clean_text(entry["content"])
        category = entry["category"]

        unified_data.append({
            "content": content,
            "category": category,
            "output_text": f"{content}"
        })

    return unified_data



## **Generación de Embeddings con FAISS**

Creamos embeddings para el contenido procesado y los indexamos utilizando FAISS.

In [4]:
def generate_embeddings(processed_data, embedding_model_name="all-MiniLM-L12-v2", index_file="indice_faiss.index"):
    """Genera embeddings para el contenido procesado e indexa con FAISS."""
    model = SentenceTransformer(embedding_model_name)
    contents = [entry.get("content", "") for entry in processed_data]

    # Verifica entradas vacías
    valid_entries = [(i, content) for i, content in enumerate(contents) if content.strip()]
    if not valid_entries:
        print("No hay contenido válido para generar embeddings.")
        return None, None

    indices, valid_contents = zip(*valid_entries)
    embeddings = model.encode(list(valid_contents), convert_to_numpy=True)

    # Crear el índice FAISS
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)

    # Guardar el índice y devolverlo
    faiss.write_index(index, index_file)
    print(f"Índice FAISS guardado en {index_file}")
    return index, embeddings


## **Proceso de Fine-Tuning**



In [5]:
def prepare_fine_tuning_dataset(unified_data):
    """
    Prepares a dataset compatible with the causal language model format.
    """
    dataset = Dataset.from_list([
        {
            "text": f"Tema: {entry['category']}\nRespuesta: {entry['content']}"
        }
        for entry in unified_data
    ])
    return dataset.train_test_split(test_size=0.1)


class BloomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        Custom compute_loss method to debug inputs passed to the model.
        """
        return super().compute_loss(model, inputs, return_outputs=return_outputs)

class EarlyStoppingCallback(TrainerCallback):
    """
    Callback personalizado para implementar early stopping basado en la pérdida de validación.
    """
    def __init__(self, patience=2):
        """
        Args:
            patience (int): Número de épocas sin mejora en la pérdida de validación antes de detener el entrenamiento.
        """
        self.patience = patience
        self.best_loss = float("inf")
        self.epochs_no_improve = 0
        self.best_model_checkpoint = None

    def on_evaluate(self, args, state, control, **kwargs):
        """
        Se ejecuta después de cada evaluación del modelo.
        """
        metrics = kwargs.get("metrics", {})
        validation_loss = metrics.get("eval_loss")
        if validation_loss is not None:
            if validation_loss < self.best_loss:
                self.best_loss = validation_loss
                self.epochs_no_improve = 0
                self.best_model_checkpoint = state.best_model_checkpoint
                print(f"Mejor pérdida de validación encontrada: {self.best_loss:.4f}")
            else:
                self.epochs_no_improve += 1
                print(f"Pérdida de validación no mejoró. Épocas sin mejora: {self.epochs_no_improve}/{self.patience}")

            # Detener si se supera la paciencia
            if self.epochs_no_improve >= self.patience:
                print("Deteniendo el entrenamiento por early stopping.")
                control.should_training_stop = True


def fine_tune_model(base_model_name, dataset, output_dir):
    """
    Fine-tunes the BLOOM model for causal language modeling and applies optimizations.
    """
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    model = AutoModelForCausalLM.from_pretrained(base_model_name, trust_remote_code=True)

    def tokenize_function(example):
        inputs = tokenizer(
            example["text"],
            padding="max_length",
            truncation=True,
            max_length=512
        )
        inputs["labels"] = inputs["input_ids"].copy()
        return inputs

    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
        return_tensors="pt"
    )

    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=4,
        eval_accumulation_steps=4,
        num_train_epochs=8,
        fp16=True,
        save_steps=50,
        save_total_limit=1,
        weight_decay=0.01,
        logging_dir="./logs",
        load_best_model_at_end=True,
        save_strategy="epoch",
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        push_to_hub=True,
        hub_private_repo=False,
        hub_token=userdata.get("HF_TOKEN")
    )

    # Añadir el callback de early stopping
    early_stopping_callback = EarlyStoppingCallback(patience=3)

    trainer = BloomTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        data_collator=data_collator,
        tokenizer=tokenizer,
        callbacks=[early_stopping_callback]
    )

    trainer.train()

    # Guardar el modelo final (mejor modelo según la validación)
    model_id = "bloom-560-finetuned-owasp-8epochs"
    trainer.save_model(output_dir)
    trainer.push_to_hub()
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    trainer.push_to_hub()
    print(f"Modelo fine-tuned guardado en {output_dir} (Mejor época).")



## **Búsqueda de Contexto con FAISS y TF-IDF**

Realizamos una búsqueda en el índice FAISS para recuperar documentos relevantes y optimizamos el contexto con TF-IDF.


In [6]:
def search_with_faiss(query, index, processed_data, embedding_model_name="all-MiniLM-L12-v2", top_k=3):
    """
    Busca en el índice FAISS los documentos más relevantes para una consulta.

    Args:
        query (str): La consulta en lenguaje natural.
        index (faiss.Index): Índice FAISS preconstruido.
        processed_data (list): Datos procesados que contienen contenido y categoría.
        embedding_model_name (str): Nombre del modelo para generar embeddingspatin
        top_k (int): Número de resultados relevantes a recuperar.

    Returns:
        list: Lista de resultados relevantes con su contenido, categoría y distancia.
    """
    model = SentenceTransformer(embedding_model_name)
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)

    results = []
    for i in range(len(indices[0])):
        idx = indices[0][i]
        result = {
            "content": processed_data[idx].get("content", "") or processed_data[idx].get("context", ""),
            "category": processed_data[idx].get("category", "Unknown"),
            "distance": distances[0][i]
        }
        results.append(result)

    return results

def truncate_context_with_tfidf(context, query, max_tokens=512):
    """Trunca el contexto priorizando secciones más relevantes con TF-IDF."""
    if not context.strip():
        print("Contexto vacío proporcionado a TF-IDF.")
        return ""

    sections = context.split("\n")
    corpus = [query] + sections
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    query_vector = tfidf_matrix[0]
    section_vectors = tfidf_matrix[1:]
    similarities = section_vectors.dot(query_vector.T).toarray().flatten()

    sorted_indices = np.argsort(similarities)[::-1]
    sorted_sections = [sections[i] for i in sorted_indices]

    truncated_context = []
    token_count = 0
    for section in sorted_sections:
        tokens = section.split()
        if token_count + len(tokens) <= max_tokens:
            truncated_context.append(section)
            token_count += len(tokens)
        else:
            break

    return "\n".join(truncated_context)


def ensure_context(context, query, processed_data, max_tokens=512):
    """
    Garantiza que el contexto no esté vacío después del truncamiento.

    Args:
        context (str): Contexto inicial.
        query (str): Consulta en lenguaje natural.
        processed_data (list): Lista de datos procesados.
        max_tokens (int): Límite máximo de tokens.

    Returns:
        str: Contexto asegurado y optimizado.
    """
    if not context.strip():
        print("Contexto vacío, utilizando contenido predeterminado.")
        fallback_context = "\n".join([entry["content"] for entry in processed_data[:3]])
        return truncate_context_with_tfidf(fallback_context, query, max_tokens=max_tokens)
    return context

## **Generación de Respuestas**

Genera respuestas utilizando el modelo fine-tuned y el contexto optimizado.


In [7]:
def truncate_to_last_sentence(text):
    """
    Trunca el texto generado hasta el último punto completo.

    Args:
        text (str): Texto generado por el modelo.

    Returns:
        str: Texto truncado hasta el último punto.
    """
    last_period = text.rfind(".")
    if last_period != -1:
        return text[:last_period + 1]
    return text

def generate_response(query, context, model_path):
    """
    Genera una respuesta usando el modelo fine-tuned.

    Args:
        query (str): La pregunta en lenguaje natural.
        context (str): Contexto recuperado y optimizado.
        model_path (str): Ruta al modelo fine-tuned.

    Returns:
        tuple: Respuesta generada y tiempo de inferencia.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)
    text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

    prompt = (
        f"Pregunta: {query}\n"
        f"Contexto: {context}\n\n"
        f"Respuesta:"
    )

    # Registrar el tiempo de inicio de la inferencia
    start_time = time.time()

    # Generar la respuesta
    result = text_gen_pipeline(prompt, max_new_tokens=150, do_sample=True, temperature=0.4)
    raw_response = result[0]["generated_text"]

    # Registrar el tiempo de fin de la inferencia
    end_time = time.time()
    inference_time = end_time - start_time

    # Extraer solo la parte relevante de la respuesta
    if "Respuesta:" in raw_response:
        response = raw_response.split("Respuesta:")[-1].strip()
    else:
        response = raw_response.strip()

    # Truncar la respuesta al último punto completo
    response = truncate_to_last_sentence(response)
    return response, inference_time

def test_model_inference_with_faiss(query, index, processed_data, model_path, embedding_model_name="all-MiniLM-L12-v2", top_k=3):
    """
    Prueba la inferencia del modelo usando el contexto generado por FAISS.

    Args:
        query (str): La consulta en lenguaje natural.
        index (faiss.Index): Índice FAISS preconstruido.
        processed_data (list): Datos procesados para recuperar contenido.
        model_path (str): Ruta al modelo para inferencia.
        embedding_model_name (str): Nombre del modelo para embeddings.
        top_k (int): Número de documentos relevantes a recuperar.

    Returns:
        tuple: Respuesta generada, contexto utilizado y tiempo de inferencia.
    """
    # Recuperar contexto con FAISS
    search_results = search_with_faiss(query, index, processed_data, embedding_model_name, top_k)
    full_context = "\n".join([result["content"] for result in search_results])
    full_context = ensure_context(full_context, query, processed_data)

    # Prueba de inferencia
    response, inference_time = generate_response(query, full_context, model_path)
    return response, full_context, inference_time

## **Ejecución Principal**

Procesamos los archivos, generamos embeddings, realizamos búsquedas y generamos respuestas.

In [8]:
def main():
    # Cargar datasets
    qa_dataset = load_json("owasp_qa_dataset_es_cleaned.json")
    faiss_dataset = load_json("owasp_pretrained_dataset_faiss.json")

    # Unificar y limpiar los datasets
    unified_data = unify_datasets(qa_dataset, faiss_dataset)
    save_json(unified_data, OUTPUT_JSON)

    # Generar embeddings e indexar
    index, _ = generate_embeddings(unified_data, index_file=INDEX_FILE)

    # Validar si el índice FAISS se generó correctamente
    if not index:
        print("Índice FAISS no se pudo generar. Abortando.")
        return

    # Preparar y realizar fine-tuning
    dataset = prepare_fine_tuning_dataset(unified_data)
    fine_tune_model(MODEL_NAME, dataset, FINE_TUNED_MODEL_PATH)

    # Consulta de ejemplo
    query = "¿Qué es el control de acceso roto?"

    # Modelo finetuneado
    original_response, original_context, original_time = test_model_inference_with_faiss(
        query, index, unified_data, FINE_TUNED_MODEL_PATH
    )
    print("\n--- Prueba Modelo Fine-tuned ---")
    print(f"Respuesta:\n{original_response}")
    print(f"Tiempo de inferencia: {original_time:.2f}s")

if __name__ == "__main__":
    main()


Índice FAISS guardado en ./indice_faiss.index


Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-359784b38c74>:108: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `BloomTrainer.__init__`. Use `processing_class` instead.
  trainer = BloomTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: p-dazad (p-dazad-universidad-de-los-andes). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,No log,1.542587
2,No log,0.997498
3,No log,0.888237
4,No log,0.937568
5,No log,0.996201
6,No log,1.113358


...

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


Modelo fine-tuned guardado en ./fine_tuned_bloom_owasp (Mejor época).


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.



--- Prueba Modelo Fine-tuned ---
Respuesta:
El control de acceso implementa el cumplimiento de política de modo que los usuarios no pueden actuar fuera de los permisos que le fueron asignados. Las fallas generalmente conducen a la divulgación de información no autorizada, la modificación o la destrucción de todos los datos o la ejecución de una función de negocio fuera de los límites del usuario. Las fallas generalmente conducen a la divulgación de información no autorizada, la modificación o la destrucción de todos los datos o la ejecución de una función de negocio fuera de los límites del usuario. El control de acceso implementa el cumplimiento de política de modo que los usuarios no pueden actuar fuera de los permisos que le fueron asignados.
Tiempo de inferencia: 10.60s


## **Exportación del Modelo a ONNX**
En este paso, se exporta el modelo de Hugging Face al formato ONNX, optimizando su ejecución en diferentes entornos y hardware especializado. Esto mejora el rendimiento y facilita su uso en producción

In [9]:
pip install optimum[exporters]

In [10]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer

model_checkpoint = "pdazad/fine_tuned_bloom_owasp"
save_directory = "onnx/"

# Load a model from transformers and export it to ONNX
ort_model = ORTModelForSequenceClassification.from_pretrained(model_checkpoint, export=True)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Save the onnx model and tokenizer
ort_model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

config.json:   0%|          | 0.00/784 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at pdazad/fine_tuned_bloom_owasp and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/bloom/modeling_bloom.py:68: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  base = torch.tensor(
/usr/local/lib/python3.11/dist-packages/transformers/models/bloom/modeling_bloom.py:841: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:
/usr/local/lib/python3.11/dist-packages/torch/onnx/symbolic_opset9.py:5385: UserWarning: Exporting aten::index operator of advanced indexing in opset 14 is achieved by combination of multiple ONNX operator

('onnx/tokenizer_config.json',
 'onnx/special_tokens_map.json',
 'onnx/tokenizer.json')

In [11]:
from huggingface_hub import upload_folder

# Define tu repositorio en el Hub
repo_id = "pdazad/fine_tuned_bloom_owasp"

# Sube la carpeta "onnx" como una subcarpeta en el repositorio
upload_folder(
    folder_path="onnx",
    repo_id=repo_id,
    repo_type="model",
    path_in_repo="onnx",
    commit_message="Subir carpeta onnx al repositorio"
)

model.onnx:   0%|          | 0.00/787k [00:00<?, ?B/s]

model.onnx_data:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/pdazad/fine_tuned_bloom_owasp/commit/c0f3110dd6ab59a59dc1afadcc427dab8b611e15', commit_message='Subir carpeta onnx al repositorio', commit_description='', oid='c0f3110dd6ab59a59dc1afadcc427dab8b611e15', pr_url=None, repo_url=RepoUrl('https://huggingface.co/pdazad/fine_tuned_bloom_owasp', endpoint='https://huggingface.co', repo_type='model', repo_id='pdazad/fine_tuned_bloom_owasp'), pr_revision=None, pr_num=None)